In [1]:
import pandas as pd
from sklearn.neighbors import BallTree
from sklearn.metrics import DistanceMetric
import numpy as np

In [2]:
df=pd.read_csv('uszips.csv',low_memory=False)

In [3]:
df.head()

,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone
0,601,18.18027,-66.75266,Adjuntas,PR,Puerto Rico,True,NaN,16773.0,100.5,72001,Adjuntas,"{""72001"": 98.76, ""72141"": 1.24}",Adjuntas|Utuado,72001|72141,False,False,America/Puerto_Rico
1,602,18.36075,-67.17541,Aguada,PR,Puerto Rico,True,NaN,37083.0,472.1,72003,Aguada,"{""72003"": 100}",Aguada,72003,False,False,America/Puerto_Rico
2,603,18.45744,-67.12225,Aguadilla,PR,Puerto Rico,True,NaN,45652.0,513.2,72005,Aguadilla,"{""72005"": 99.76, ""72099"": 0.24}",Aguadilla|Moca,72005|72099,False,False,America/Puerto_Rico
3,606,18.16585,-66.93716,Maricao,PR,Puerto Rico,True,NaN,6231.0,54.3,72093,Maricao,"{""72093"": 82.28, ""72153"": 11.67, ""72121"": 6.05}",Maricao|Yauco|Sabana Grande,72093|72153|72121,False,False,America/Puerto_Rico
4,610,18.29110,-67.12243,Anasco,PR,Puerto Rico,True,NaN,26502.0,275.7,72011,Añasco,"{""72011"": 96.71, ""72099"": 2.81, ""72083"": 0.37,...",Añasco|Moca|Las Marías|Aguada,72011|72099|72083|72003,False,False,America/Puerto_Rico


In [4]:
df2=df[df.state_id == 'NY']

In [5]:
df2

,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone
1839,6390,41.27027,-71.98886,Fishers Island,NY,New York,True,NaN,61.0,5.8,36103,Suffolk,"{""36103"": 100}",Suffolk,36103,False,False,America/New_York
2583,10001,40.75064,-73.99728,New York,NY,New York,True,NaN,25026.0,15489.3,36061,New York,"{""36061"": 100}",New York,36061,False,False,America/New_York
2584,10002,40.71597,-73.98692,New York,NY,New York,True,NaN,74363.0,34916.7,36061,New York,"{""36061"": 100}",New York,36061,False,False,America/New_York
2585,10003,40.73184,-73.98915,New York,NY,New York,True,NaN,54671.0,36928.8,36061,New York,"{""36061"": 100}",New York,36061,False,False,America/New_York
2586,10004,40.69431,-74.01835,New York,NY,New York,True,NaN,3310.0,2805.2,36061,New York,"{""36061"": 100}",New York,36061,False,False,America/New_York
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4403,14898,42.05729,-77.43482,Woodhull,NY,New York,True,NaN,1772.0,11.2,36101,Steuben,"{""36101"": 100}",Steuben,36101,False,False,America/New_York
4404,14901,42.07363,-76.74137,Elmira,NY,New York,True,NaN,15581.0,181.2,36015,Chemung,"{""36015"": 100}",Chemung,36015,False,False,America/New_York
4405,14903,42.12372,-76.87771,Elmira,NY,New York,True,NaN,7186.0,128.2,36015,Chemung,"{""36015"": 100}",Chemung,36015,False,False,America/New_York
4406,14904,42.07007,-76.80798,Elmira,NY,New York,True,NaN,15109.0,904.6,36015,Chemung,"{""36015"": 100}",Chemung,36015,False,False,America/New_York


In [6]:
df2=df2[['zip','lat','lng']]

In [7]:
df2.sample(50)

,zip,lat,lng
3301,12444,42.24942,-74.28993
3300,12443,41.93388,-74.08536
3888,13652,44.43579,-75.21338
2806,10964,41.01642,-73.91431
3052,11787,40.85338,-73.21089
3751,13328,42.98247,-75.42836
3453,12770,41.44681,-74.85028
3575,12955,44.78934,-73.95855
4272,14709,42.35279,-77.98267
3236,12189,42.73618,-73.71688


In [8]:
lats=pd.read_csv('NYC_listings_Clean2.csv',low_memory=False)

In [9]:

# DF1
coords = np.radians(df2[['lat', 'lng']])
dist = DistanceMetric.get_metric('haversine')
tree = BallTree(coords, metric=dist)

# DF2
coords = np.radians(lats[['latitude', 'longitude']])
distances, indices = tree.query(coords, k=1)
lats['zip'] = df2['zip'].iloc[indices.flatten()].values


In [10]:
lats


,Unnamed: 0,id,host_id,host_listings_count,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bedrooms,beds,price,minimum_nights,availability_30,number_of_reviews,review_scores_rating,zip
0,0,2539,2787,9.0,Kensington,40.645290,-73.972380,Private room in rental unit,Private room,2,1.0,1.0,$299.00,30,21,9,4.89,11218
1,1,2595,2845,6.0,Midtown,40.753560,-73.985590,Entire rental unit,Entire home/apt,1,NaN,1.0,$175.00,30,0,49,4.68,10110
2,2,5121,7356,2.0,Bedford-Stuyvesant,40.685350,-73.955120,Private room in rental unit,Private room,2,1.0,1.0,$60.00,30,5,50,4.52,11216
3,3,45910,204539,7.0,Ridgewood,40.703090,-73.899630,Entire townhouse,Entire home/apt,16,5.0,10.0,$425.00,30,30,13,4.42,11385
4,4,5136,7378,1.0,Sunset Park,40.662650,-73.994540,Entire rental unit,Entire home/apt,4,2.0,2.0,$275.00,21,0,3,5.00,11215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39876,39876,43728504,23049030,5.0,Rockaway Park,40.581060,-73.829850,Entire home,Entire home/apt,2,1.0,1.0,$157.00,3,0,33,4.91,11694
39877,39877,44578886,48009648,1.0,Upper West Side,40.784590,-74.012530,Entire rental unit,Entire home/apt,4,1.0,2.0,$108.00,3,0,52,4.79,10069
39878,39878,46355090,371552251,1.0,Bellerose,40.723410,-73.722350,Entire rental unit,Entire home/apt,2,1.0,2.0,$140.00,3,0,11,4.45,11426
39879,39879,698195550745703156,151487807,1.0,Williamsburg,40.709192,-73.970121,Entire rental unit,Entire home/apt,4,2.0,2.0,$400.00,30,0,0,NaN,11249


In [11]:
lats.to_csv('listings_with_zips.csv')